# pandas的拼接操作

pandas的拼接分为两种：
- 级联：pd.concat, pd.append
- 合并：pd.merge, pd.join

## 0. 回顾numpy的级联

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

In [6]:
n1 = np.random.randint(0,10,size=(3,3))
n2 = np.random.randint(10,20,size=(3,2))
np.concatenate((n1,n2),axis=1)

array([[ 4,  0,  4, 14, 17],
       [ 5,  9,  2, 10, 17],
       [ 2,  3,  1, 15, 11]])

============================================

练习12：

1. 生成2个3*3的矩阵，对其分别进行两个维度上的级联

============================================

为方便讲解，我们首先定义一个生成DataFrame的函数：

In [19]:
def create_DF(index,columns):
    data = []
    for i in index:
        items = []
        for j in columns:
            s = j + i
            items.append(s)
        data.append(items)
    return DataFrame(data,index=index,columns=columns)    

In [27]:
def create_DF2(index,columns):
    return DataFrame({j:[j+i for i in index] for j in columns},index=index)

In [28]:
create_DF2(list('34567'),list('BCDEF'))

,B,C,D,E,F
3,B3,C3,D3,E3,F3
4,B4,C4,D4,E4,F4
5,B5,C5,D5,E5,F5
6,B6,C6,D6,E6,F6
7,B7,C7,D7,E7,F7


In [20]:
create_DF(list('23456'),list('ABCDE'))

,A,B,C,D,E
2,A2,B2,C2,D2,E2
3,A3,B3,C3,D3,E3
4,A4,B4,C4,D4,E4
5,A5,B5,C5,D5,E5
6,A6,B6,C6,D6,E6


## 1. 使用pd.concat()级联

In [29]:
df1 = create_DF(list('12345'),list('ABCDE'))
df2 = create_DF(list('12345'),list('BCDEF'))

In [31]:
pd.concat((df1,df2),axis=1)

,A,B,C,D,E,B,C,D,E,F
1,A1,B1,C1,D1,E1,B1,C1,D1,E1,F1
2,A2,B2,C2,D2,E2,B2,C2,D2,E2,F2
3,A3,B3,C3,D3,E3,B3,C3,D3,E3,F3
4,A4,B4,C4,D4,E4,B4,C4,D4,E4,F4
5,A5,B5,C5,D5,E5,B5,C5,D5,E5,F5


In [33]:
df3 = create_DF(list('123'),list('CDE'))
pd.concat((df1,df3),axis=1)

,A,B,C,D,E,C,D,E
1,A1,B1,C1,D1,E1,C1,D1,E1
2,A2,B2,C2,D2,E2,C2,D2,E2
3,A3,B3,C3,D3,E3,C3,D3,E3
4,A4,B4,C4,D4,E4,NaN,NaN,NaN
5,A5,B5,C5,D5,E5,NaN,NaN,NaN


In [ ]:
# pandas中，级联允许形状不同，缺失的索引补充NaN

pandas使用pd.concat函数，与np.concatenate函数类似，只是多了一些参数：
```
objs
axis=0
join='outer'
join_axes=None
ignore_index=False
```

In [35]:
# 索引没有特定含义的时候，可以使用如下方法处理索引重复的问题
pd.concat((df1,df2),ignore_index=True)

,A,B,C,D,E,F
0,A1,B1,C1,D1,E1,NaN
1,A2,B2,C2,D2,E2,NaN
2,A3,B3,C3,D3,E3,NaN
3,A4,B4,C4,D4,E4,NaN
4,A5,B5,C5,D5,E5,NaN
5,NaN,B1,C1,D1,E1,F1
6,NaN,B2,C2,D2,E2,F2
7,NaN,B3,C3,D3,E3,F3
8,NaN,B4,C4,D4,E4,F4
9,NaN,B5,C5,D5,E5,F5


In [36]:
# 索引有特定含义，可以引入keys参数，对两张表做分区说明
pd.concat((df1,df2),keys=['上学期','下学期'])

A   B   C   D   E    F
上学期 1   A1  B1  C1  D1  E1  NaN
    2   A2  B2  C2  D2  E2  NaN
    3   A3  B3  C3  D3  E3  NaN
    4   A4  B4  C4  D4  E4  NaN
    5   A5  B5  C5  D5  E5  NaN
下学期 1  NaN  B1  C1  D1  E1   F1
    2  NaN  B2  C2  D2  E2   F2
    3  NaN  B3  C3  D3  E3   F3
    4  NaN  B4  C4  D4  E4   F4
    5  NaN  B5  C5  D5  E5   F5

In [41]:
# 应该更多的采用外连接来级联，保留更多的原始数据
# inner内连接，取交集
# outer外连接, 取并集
pd.concat((df1,df2),join='outer')

,A,B,C,D,E,F
1,A1,B1,C1,D1,E1,NaN
2,A2,B2,C2,D2,E2,NaN
3,A3,B3,C3,D3,E3,NaN
4,A4,B4,C4,D4,E4,NaN
5,A5,B5,C5,D5,E5,NaN
1,NaN,B1,C1,D1,E1,F1
2,NaN,B2,C2,D2,E2,F2
3,NaN,B3,C3,D3,E3,F3
4,NaN,B4,C4,D4,E4,F4
5,NaN,B5,C5,D5,E5,F5


In [46]:
pd.concat((df1,df2),join_axes=[df2.columns])

,B,C,D,E,F
1,B1,C1,D1,E1,NaN
2,B2,C2,D2,E2,NaN
3,B3,C3,D3,E3,NaN
4,B4,C4,D4,E4,NaN
5,B5,C5,D5,E5,NaN
1,B1,C1,D1,E1,F1
2,B2,C2,D2,E2,F2
3,B3,C3,D3,E3,F3
4,B4,C4,D4,E4,F4
5,B5,C5,D5,E5,F5


In [47]:
type(df2.columns)

pandas.core.indexes.base.Index

In [48]:
pd.core.indexes.base.Index?

In [56]:
index = pd.Index(['1','2'])

In [57]:
# 指定连接轴进行级联
pd.concat((df1,df2),join_axes=[index],axis=1)

,A,B,C,D,E,B,C,D,E,F
1,A1,B1,C1,D1,E1,B1,C1,D1,E1,F1
2,A2,B2,C2,D2,E2,B2,C2,D2,E2,F2


### 1)  简单级联

和np.concatenate一样，优先增加行数（默认axis=0）

可以通过设置axis来改变级联方向

级连会把该方向上索引相同的元素放在一行（一列），index/columns在级联时可以重复

也可以选择忽略ignore_index，重新索引

或者使用多层索引 keys  

concat([x,y],keys=['x','y'])

============================================

练习13：

1. 想一想级联的应用场景？

2. 使用昨天的知识，建立一个期中考试张三、李四的成绩表ddd

3. 假设新增考试学科"计算机"，如何实现？

4. 新增王老五同学的成绩，如何实现？

============================================

In [62]:
score = DataFrame(data = np.random.randint(0,150,size=(2,3)),
                 index = ['张三','李四'],
                 columns = ['python','java','c'])
score

,python,java,c
张三,86,111,26
李四,107,27,0


In [66]:
score['java'] = [98,76]
score

,python,java,c
张三,86,98,26
李四,107,76,0


In [59]:
score['计算机'] = [100,90]

In [60]:
score

,python,java,c,计算机
张三,82,69,78,100
李四,47,92,37,90


In [63]:
# 使用级联处理
s = Series(data=[100,99],index=['张三','李四'],name='计算机')
# pandas可以直接跟series对象进行级联
pd.concat((score,s),axis=1)

,python,java,c,计算机
张三,86,111,26,100
李四,107,27,0,99


In [64]:
df = DataFrame(s)
df

,计算机
张三,100
李四,99


In [75]:
result = pd.concat((score,df),axis=1)

In [76]:
result

,python,java,c,计算机
张三,86,98,26,100
李四,107,76,0,99


In [78]:
df2 = DataFrame(data=[[100,98,78,56]],index=['王老五'],columns=result.columns)
df2

,python,java,c,计算机
王老五,100,98,78,56


In [80]:
result1 = pd.concat((result,df2))

In [82]:
# 使用索引的方式进行添加
result1.loc['赵小六'] = [23,55,78,98]

In [83]:
result1

,python,java,c,计算机
张三,86,98,26,100
李四,107,76,0,99
王老五,100,98,78,56
赵小六,23,55,78,98


### 2) 不匹配级联

不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致

有3种连接方式：

- 外连接：补NaN（默认模式）

- 内连接：只连接匹配的项

- 连接指定轴 join_axes

============================================

练习14：

    假设【期末】考试ddd2的成绩没有张三的，只有李四、王老五、赵小六的，使用多种方法级联

============================================

### 3) 使用append()函数添加

由于在后面级联的使用非常普遍，因此有一个函数append专门用于在后面添加

注意:append函数只是沿着axis=0的方向进行级联

In [85]:
result

,python,java,c,计算机
张三,86,98,26,100
李四,107,76,0,99


In [84]:
df2

,python,java,c,计算机
王老五,100,98,78,56


In [87]:
result.append(df2)

,python,java,c,计算机
张三,86,98,26,100
李四,107,76,0,99
王老五,100,98,78,56


============================================

练习15：

    新建一个只有张三李四王老五的期末考试成绩单ddd3，使用append()与期中考试成绩表ddd级联

============================================

## 2. 使用pd.merge()合并

merge与concat的区别在于，merge需要依据某一共同列来进行合并

使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。

注意每一列元素的顺序不要求一致

In [89]:
table1 = pd.read_excel('关系表.xlsx',sheet_name=1)
table2 = pd.read_excel('关系表.xlsx',sheet_name=2)
table3 = pd.read_excel('关系表.xlsx',sheet_name=3)
table4 = pd.read_excel('关系表.xlsx',sheet_name=4)

In [90]:
display(table1,table2,table3,table4)

,手机型号,参考价格
0,windowsPhone,2500
1,iPhone,7500
2,Android,4000


,手机型号,重量
0,windowsPhone,0.50
1,iPhone,0.40
2,Android,0.45
3,other,0.60


,经销商,发货地区,手机型号
0,dancer,beijing,iPhone
1,lucy,beijing,Android
2,tom,guangzhou,iPhone
3,petter,shenzhen,windowsPhone
4,mery,guangzhou,Android


,发货地区,手机型号,价格
0,beijing,iPhone,7000
1,beijing,windowsPhone,2300
2,beijing,Android,3600
3,guangzhou,iPhone,7600
4,guangzhou,windowsPhone,2800
5,guangzhou,Android,4200
6,shenzhen,iPhone,7400
7,shenzhen,windowsPhone,2750
8,shenzhen,Android,3900


###  1) 一对一合并

### 2) 多对一合并

### 3) 多对多合并

### 4) key的规范化

In [94]:
# 1. 如果两个表中，只有一列内容是有重复的，就参考这一列进行合并
# 2. 默认的合并方式，只取交集
# 3. how 设置合并的参考列
# 4. 合并就是以列为参考的，不是以行
pd.merge(table1,table2,how='right')

,手机型号,参考价格,重量
0,windowsPhone,2500.0,0.50
1,iPhone,7500.0,0.40
2,Android,4000.0,0.45
3,other,NaN,0.60


- 使用on=显式指定哪一列为key,当有多个key相同时使用

In [95]:
# 默认按照所有的相同列为合并参考
pd.merge(table3,table4)

,经销商,发货地区,手机型号,价格
0,dancer,beijing,iPhone,7000
1,lucy,beijing,Android,3600
2,tom,guangzhou,iPhone,7600
3,petter,shenzhen,windowsPhone,2750
4,mery,guangzhou,Android,4200


In [96]:
pd.merge(table3,table4,on='手机型号')

,经销商,发货地区_x,手机型号,发货地区_y,价格
0,dancer,beijing,iPhone,beijing,7000
1,dancer,beijing,iPhone,guangzhou,7600
2,dancer,beijing,iPhone,shenzhen,7400
3,tom,guangzhou,iPhone,beijing,7000
4,tom,guangzhou,iPhone,guangzhou,7600
5,tom,guangzhou,iPhone,shenzhen,7400
6,lucy,beijing,Android,beijing,3600
7,lucy,beijing,Android,guangzhou,4200
8,lucy,beijing,Android,shenzhen,3900
9,mery,guangzhou,Android,beijing,3600


In [116]:
pd.merge(table3,table4,on='发货地区',suffixes=['_A','_B'])

,经销商,发货地区,手机型号_A,手机型号_B,价格
0,dancer,beijing,iPhone,iPhone,7000
1,dancer,beijing,iPhone,windowsPhone,2300
2,dancer,beijing,iPhone,Android,3600
3,lucy,beijing,Android,iPhone,7000
4,lucy,beijing,Android,windowsPhone,2300
5,lucy,beijing,Android,Android,3600
6,tom,guangzhou,iPhone,iPhone,7600
7,tom,guangzhou,iPhone,windowsPhone,2800
8,tom,guangzhou,iPhone,Android,4200
9,mery,guangzhou,Android,iPhone,7600


In [101]:
pd.merge(table3,table4,on=['手机型号','发货地区'],how='inner')

,经销商,发货地区,手机型号,价格
0,dancer,beijing,iPhone,7000
1,lucy,beijing,Android,3600
2,tom,guangzhou,iPhone,7600
3,petter,shenzhen,windowsPhone,2750
4,mery,guangzhou,Android,4200


- 使用left_on和right_on指定左右两边的列作为key，当左右两边的key都不想等时使用

In [102]:
table5 = table1.copy()
table5.columns = ['型号','参考价格']
table5

,型号,参考价格
0,windowsPhone,2500
1,iPhone,7500
2,Android,4000


In [106]:
# 两张表中不存在相同的列表签，可以使用left_on和right_on来显示制定合并参考列
# 合并之后两列都会保留，可以使用drop函数删除没用的列
pd.merge(table2,table5,left_on='手机型号',right_on='型号').drop(labels=['型号'],axis=1)

,手机型号,重量,参考价格
0,windowsPhone,0.50,2500
1,iPhone,0.40,7500
2,Android,0.45,4000


In [113]:
# 设置一列内容为dataFrame的索引
table6 = table1.set_index('手机型号')
table6

,参考价格
手机型号,
windowsPhone,2500
iPhone,7500
Android,4000


In [115]:
# 以表的行索引为合并参考
# 设置left_index或right_index为True
pd.merge(table2,table6,left_on='手机型号',right_index=True)

,手机型号,重量,参考价格
0,windowsPhone,0.50,2500
1,iPhone,0.40,7500
2,Android,0.45,4000


============================================

练习16：

1. 假设有两份成绩单，除了ddd是张三李四王老五之外，还有ddd4是张三和赵小六的成绩单，如何合并？

2. 如果ddd4中张三的名字被打错了，成为了张十三，怎么办？

3. 自行练习多对一，多对多的情况  

4. 自学left_index,right_index

============================================

In [142]:
df1 = DataFrame(data=[[90,95,78],[78,89,90],[90,90,100]],index=['张三','李四','王老五'],
               columns=['python','c','java'])
df1

,python,c,java
张三,90,95,78
李四,78,89,90
王老五,90,90,100


In [141]:
df2 = DataFrame(data=[[90,95,78],[99,70,69]],index=['张十三','赵小六'],
               columns=['语文','数学','英语'])
df2

,语文,数学,英语
张十三,90,95,78
赵小六,99,70,69


In [140]:
pd.merge(df1,df2,left_index=True,right_index=True,how='outer')

,python,c,java,语文,数学,英语
张三,90.0,95.0,78.0,90.0,95.0,78.0
李四,78.0,89.0,90.0,NaN,NaN,NaN
王老五,90.0,90.0,100.0,NaN,NaN,NaN
赵小六,NaN,NaN,NaN,99.0,70.0,69.0


In [143]:
pd.merge(df1,df2,left_index=True,right_index=True,how='outer')

,python,c,java,语文,数学,英语
张三,90.0,95.0,78.0,NaN,NaN,NaN
张十三,NaN,NaN,NaN,90.0,95.0,78.0
李四,78.0,89.0,90.0,NaN,NaN,NaN
王老五,90.0,90.0,100.0,NaN,NaN,NaN
赵小六,NaN,NaN,NaN,99.0,70.0,69.0


In [197]:
df3 = DataFrame(data=[[100,100,100],[90,80,70],[80,50,50]],
               columns = ['张三','李四','王五'],dtype=object)
df3

,张三,李四,王五
0,100,100,100
1,90,80,70
2,80,50,50


In [198]:
df4 = DataFrame(data=[[100,50],[90,60],[80,70]],columns=['张十三','赵小六'],dtype=object)
df4

,张十三,赵小六
0,100,50
1,90,60
2,80,70


In [199]:
df3

,张三,李四,王五
0,100,100,100
1,90,80,70
2,80,50,50


In [200]:
df4

,张十三,赵小六
0,100,50
1,90,60
2,80,70


In [201]:
pd.merge(df3,df4,left_on='张三',right_on='张十三')

,张三,李四,王五,张十三,赵小六
0,100,100,100,100,50
1,90,80,70,90,60
2,80,50,50,80,70


In [ ]:
# 数值型数据，尽量采用级联而不是合并
# 字符串型数据，可以使用合并

# 一旦数值出现重复值，就会导致业务逻辑变成1对多或者对对多的关系

### 5) 内合并与外合并

- 内合并：只保留两者都有的key（默认模式）

- 外合并 how='outer'：补NaN

- 左合并、右合并：how='left'，how='right'，

============================================

练习17：



1. 考虑应用情景，使用多种方式合并ddd与ddd4

============================================

### 6) 列冲突的解决

当列冲突时，即有多个列名称相同时，需要使用on=来指定哪一个列作为key，配合suffixes指定冲突列名

可以使用suffixes=自己指定后缀

============================================

练习18：

    假设有两个同学都叫李四，ddd5、ddd6都是张三和李四的成绩表，如何合并？

============================================

## 作业
## 3. 案例分析：美国各州人口数据分析

作业知识补充

首先导入文件，并查看数据样本

In [2]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

In [4]:
abbrevs=pd.read_csv('../data/state-abbrevs.csv')
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [7]:
pop = pd.read_csv('../data/state-population.csv')

In [9]:
abbr_merge = pd.merge(abbrevs,pop,left_on='abbreviation',right_on="state/region",how='outer')
abbr_merge.head()

,state,abbreviation,state/region,ages,year,population
0,Alabama,AL,AL,under18,2012,1117489.0
1,Alabama,AL,AL,total,2012,4817528.0
2,Alabama,AL,AL,under18,2010,1130966.0
3,Alabama,AL,AL,total,2010,4785570.0
4,Alabama,AL,AL,under18,2011,1125763.0


合并pop与abbrevs两个DataFrame，分别依据state/region列和abbreviation列来合并。

为了保留所有信息，使用外合并。

去除abbreviation的那一列（axis=1）

In [10]:
abbr_merge_cp = abbr_merge.copy()
abbr_merge_cp = abbr_merge_cp.drop(labels=['abbreviation'],axis=1)
abbr_merge_cp

,state,state/region,ages,year,population
0,Alabama,AL,under18,2012,1117489.0
1,Alabama,AL,total,2012,4817528.0
2,Alabama,AL,under18,2010,1130966.0
3,Alabama,AL,total,2010,4785570.0
4,Alabama,AL,under18,2011,1125763.0


查看存在缺失数据的列。

使用.isnull().any()，只有某一列存在一个缺失数据，就会显示True。

In [56]:
a=abbr_merge_cp.isnull().any()
lose_comm=abbr_merge_cp.columns[a]
lose_comm

Index(['state', 'population'], dtype='object')

查看缺失数据

In [59]:
abbr_merge_cp[lose_comm]

,state,population
0,Alabama,1117489.0
1,Alabama,4817528.0
2,Alabama,1130966.0
3,Alabama,4785570.0
4,Alabama,1125763.0
5,Alabama,4801627.0
6,Alabama,4757938.0
7,Alabama,1134192.0
8,Alabama,1111481.0
9,Alabama,4833722.0


根据数据是否缺失情况显示数据，如果缺失为True，那么显示

找到有哪些state/region使得state的值为NaN，使用unique()查看非重复值

为找到的这些state/region的state项补上正确的值，从而去除掉state这一列的所有NaN！

记住这样清除缺失数据NaN的方法！

合并各州面积数据areas，使用左合并。

思考一下为什么使用外合并？



继续寻找存在缺失数据的列

我们会发现area(sq.mi)这一列有缺失数据，为了找出是哪一行，我们需要找出是哪个state没有数据

去除含有缺失数据的行

查看数据是否缺失

找出2010年的全民人口数据,df.query(查询语句)

对查询结果进行处理，以state列作为新的行索引:set_index

计算人口密度。注意是Series/Series，其结果还是一个Series。

排序，并找出人口密度最高的五个州sort_values()

找出人口密度最低的五个州

要点总结：
- 统一用loc()索引
- 善于使用.isnull().any()找到存在NaN的列
- 善于使用.unique()确定该列中哪些key是我们需要的
- 一般使用外合并、左合并，目的只有一个：宁愿该列是NaN也不要丢弃其他列的信息

## 回顾：Series/DataFrame运算与ndarray运算的区别

- Series与DataFrame没有广播，如果对应index没有值，则记为NaN；或者使用add的fill_value来补缺失值
- ndarray有广播，通过重复已有值来计算